# Build Futures 

### Overview

NOTE: This notebook needs an active Bloomberg connection to run. This task was discussed with professor and done in the Financial Mathematics computer lab.

The data output is a mulit-index dataframe with index levels - Commodity, Contract number, Date

Covers all the data as per availability for each commodity from 1st to expire upto 12th to expire. Some commodity codes were adjusted for BBG versus the paper (for example: Aluminum has Bloomberg symbol as "LA" versue "AL" in the paper). 

Total commodities - 31

### Challenges

Data

- Commodities which have only 1st to expire contracts and no other contract data available - Barley, Coal, Propane, Broilers
- Commodity which has data starting after Dec 2008 - Butter
- For most commodities we are able to fetch data upto 12th to expire contract, however some commodities do not have data so further out. For all the details, see the end of the notebook.

Bloomberg

- Data download from BBG API is inconsistent and sporadic depending on the size of the data pull. The notebook has been setup in such a way, to try redownloading if a ticker missed out on downloading data.

FinMath Library (pandas version)

The Pandas version in FinMath library was outdated/old (1.5.3). This meant that the pickle files (original datastructure) saved from this version could not be read in our personal laptops (pandas version 2+). This was resolved by saving the files in .csv format instead which could be used independent of the Pandas version.

### Note on commodity futures

Each commodity future contract starts trading approximately 2-2.5 years before its scheduled expiry. So at any point in time, each commodity can have upto 20 live contracts being traded on the exchange. These contracts usually roll on a monthly basis, depening on the contract specifications (see Bloomberg <DES> for more details).    

However, the liquidity is only really significant before 6-12 months (something the paper also accounts for). Bloomberg already accounts for the rolling of contracts and constructs a time series for each commodity with respect to the expiry. For example, this can be checked as 'NG1 Comdty' for 1st to expire Natural Gas contracts. 

For backtest and research purpose, we use these time series downloaded from Bloomberg for our project.

### Note on roll adjustment

The price-adjustment for rolling of the contracts has been done. We apply ratio-adjusted methodology.

In [5]:
import pandas as pd
import numpy as np
from xbbg import blp

# Futures Contracts

### Mapping of each commodity with their Bloomberg symbols

In [2]:
commodities = {
    'Barley': 'WA',
    'Butter': 'BUT',
    'Canola': 'RS',
    'Cocoa' : 'CC',
    'Coffee': 'KC',
    'Corn' : 'C ',
    'Cotton': 'CT',
    'Lumber': 'LB',
    'Oats': 'O ',
    'Orange juice': 'JO',
    'Rough rice': 'RR',
    'Soybean meal': 'SM',
    'Soybeans': 'S ',
    'Wheat': 'W ',
    'Crude Oil': 'CL',
    'Gasoline': 'XB',
    'Heating Oil': 'HO',
    'Natural gas': 'NG',
    'Propane': 'PN',
    'Unleaded gas': 'HU',
    'Broilers': 'AH',
    'Feeder cattle': 'FC',
    'Lean hogs': 'LH',
    'Live cattle': 'LC',
    'Aluminium': 'LA',
    'Coal': 'XW',
    'Copper': 'HG',
    'Gold': 'GC',
    'Palladium': 'PA',
    'Platinum': 'PL',
    'Silver': 'SI'
}

Initial variables

In [3]:
TYPE = 'Comdty'
ADJ = 'B:00_0_R'

STARTDATE = '1970-01-01'
ENDDATE = '2024-02-28'

Function to download 'PX_LAST' data from Bloomberg using API for a particular ticker, start date and end date

In [4]:
def download_data(ticker, start_date = STARTDATE, end_date = ENDDATE):
    try:
        data = blp.bdh(ticker, flds= 'PX_LAST', start_date=start_date, end_date=end_date)       
        if not data.empty and isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.droplevel(0)
        else:
            data = None
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
        data = None
    return data

Setup a dictionary for each ticker's download and then finally merging them in a single dataframe

In [5]:
data_dict = {}
failed_tickers = {}

for name, ticker_base in commodities.items():
    for i in range(1, 13):
        ticker = f'{ticker_base}{i} B:00_0_R Comdty'
        data = download_data(ticker)
        if data is not None:
            data_dict[(name, i)] = data
        else:
            failed_tickers[(name, i)] = ticker

for (name, i), ticker in failed_tickers.items():
    print(f"Retrying download for: {ticker}")
    data = download_data(ticker)
    if data is not None:
        data_dict[(name, i)] = data
    
failed_tickers = {(name, i): ticker for (name, i), ticker in failed_tickers.items() if (name, i) not in data_dict}

if data_dict:
    commodities_data = pd.concat(data_dict.values(), keys=data_dict.keys(), names=['Commodity', 'Contract', 'Date'])
else:
    commodities_data = pd.DataFrame()

Retrying download for: WA2 B:00_0_R Comdty
Retrying download for: WA3 B:00_0_R Comdty
Retrying download for: WA4 B:00_0_R Comdty
Retrying download for: WA5 B:00_0_R Comdty
Retrying download for: WA6 B:00_0_R Comdty
Retrying download for: WA7 B:00_0_R Comdty
Retrying download for: WA8 B:00_0_R Comdty
Retrying download for: WA9 B:00_0_R Comdty
Retrying download for: WA10 B:00_0_R Comdty
Retrying download for: WA11 B:00_0_R Comdty
Retrying download for: WA12 B:00_0_R Comdty
Retrying download for: RS12 B:00_0_R Comdty
Retrying download for: CC11 B:00_0_R Comdty
Retrying download for: CC12 B:00_0_R Comdty
Retrying download for: LB8 B:00_0_R Comdty
Retrying download for: LB9 B:00_0_R Comdty
Retrying download for: LB10 B:00_0_R Comdty
Retrying download for: LB11 B:00_0_R Comdty
Retrying download for: LB12 B:00_0_R Comdty
Retrying download for: RR8 B:00_0_R Comdty
Retrying download for: RR9 B:00_0_R Comdty
Retrying download for: RR10 B:00_0_R Comdty
Retrying download for: RR11 B:00_0_R Comdty


In [7]:
for (name, i), ticker in failed_tickers.items():
    print(f"Retrying download for: {ticker}")
    data = download_data(ticker)
    if data is not None:
        data_dict[(name, i)] = data
    
failed_tickers = {(name, i): ticker for (name, i), ticker in failed_tickers.items() if (name, i) not in data_dict}

if data_dict:
    commodities_data = pd.concat(data_dict.values(), keys=data_dict.keys(), names=['Commodity', 'Contract', 'Date'])
else:
    commodities_data = pd.DataFrame()

Retrying download for: WA2 B:00_0_R Comdty
Retrying download for: WA3 B:00_0_R Comdty
Retrying download for: WA4 B:00_0_R Comdty
Retrying download for: WA5 B:00_0_R Comdty
Retrying download for: WA6 B:00_0_R Comdty
Retrying download for: WA7 B:00_0_R Comdty
Retrying download for: WA8 B:00_0_R Comdty
Retrying download for: WA9 B:00_0_R Comdty
Retrying download for: WA10 B:00_0_R Comdty
Retrying download for: WA11 B:00_0_R Comdty
Retrying download for: WA12 B:00_0_R Comdty
Retrying download for: RS12 B:00_0_R Comdty
Retrying download for: CC11 B:00_0_R Comdty
Retrying download for: CC12 B:00_0_R Comdty
Retrying download for: LB8 B:00_0_R Comdty
Retrying download for: LB9 B:00_0_R Comdty
Retrying download for: LB10 B:00_0_R Comdty
Retrying download for: LB11 B:00_0_R Comdty
Retrying download for: LB12 B:00_0_R Comdty
Retrying download for: RR8 B:00_0_R Comdty
Retrying download for: RR9 B:00_0_R Comdty
Retrying download for: RR10 B:00_0_R Comdty
Retrying download for: RR11 B:00_0_R Comdty


In [8]:
commodities_data

PX_LAST
Commodity   Contract Date               
Barley      1        1991-03-25  292.470
                     1991-03-26  292.170
                     1991-03-27  290.980
                     1991-03-28  290.980
                     1991-04-01  293.970
...                                  ...
Natural gas 7        2024-02-22    2.584
                     2024-02-23    2.473
                     2024-02-26    2.531
                     2024-02-27    2.612
                     2024-02-28    2.667

[2566132 rows x 1 columns]

In [9]:
commodities_data.to_csv('commodities_data_2024.csv')

if failed_tickers:
    print("Failed to download the following tickers after retries:")
    for (name, i), ticker in failed_tickers.items():
        print(f"{name} contract {i}: {ticker}")

Failed to download the following tickers after retries:
Barley contract 2: WA2 B:00_0_R Comdty
Barley contract 3: WA3 B:00_0_R Comdty
Barley contract 4: WA4 B:00_0_R Comdty
Barley contract 5: WA5 B:00_0_R Comdty
Barley contract 6: WA6 B:00_0_R Comdty
Barley contract 7: WA7 B:00_0_R Comdty
Barley contract 8: WA8 B:00_0_R Comdty
Barley contract 9: WA9 B:00_0_R Comdty
Barley contract 10: WA10 B:00_0_R Comdty
Barley contract 11: WA11 B:00_0_R Comdty
Barley contract 12: WA12 B:00_0_R Comdty
Canola contract 12: RS12 B:00_0_R Comdty
Cocoa contract 11: CC11 B:00_0_R Comdty
Cocoa contract 12: CC12 B:00_0_R Comdty
Lumber contract 8: LB8 B:00_0_R Comdty
Lumber contract 9: LB9 B:00_0_R Comdty
Lumber contract 10: LB10 B:00_0_R Comdty
Lumber contract 11: LB11 B:00_0_R Comdty
Lumber contract 12: LB12 B:00_0_R Comdty
Rough rice contract 8: RR8 B:00_0_R Comdty
Rough rice contract 9: RR9 B:00_0_R Comdty
Rough rice contract 10: RR10 B:00_0_R Comdty
Rough rice contract 11: RR11 B:00_0_R Comdty
Rough rice 